<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/22520975_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHUẨN BỊ MÔI TRƯỜNG

In [1]:
!pip install -q datasets pyvi torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

from datasets import load_dataset
from pyvi import ViTokenizer
from torchmetrics.text.rouge import ROUGEScore
import numpy as np
import random
from collections import Counter
import time

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device đang sử dụng: {device}")

Device đang sử dụng: cuda


# TIỀN XỬ LÝ DỮ LIỆU

In [4]:
MAX_SEQ_LEN = 100
BATCH_SIZE = 32

class Vocab:
    def __init__(self, dataset_iterator, key, is_src=True, max_size=20000, min_freq=3):
        self.token2idx = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.idx2token = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.is_src = is_src

        print(f"Đang xây dựng từ điển ({'Tiếng Anh' if is_src else 'Tiếng Việt'})...")
        counter = Counter()
        for item in dataset_iterator:
            text = item[key]
            tokens = self.tokenize(text)
            counter.update(tokens)

        most_common = [token for token, freq in counter.most_common(max_size) if freq >= min_freq]
        for idx, token in enumerate(most_common, start=4):
            self.token2idx[token] = idx
            self.idx2token[idx] = token

        print(f"-> Hoàn tất. Kích thước Vocab: {len(self.token2idx)}")

    def tokenize(self, text):
        text = str(text).lower().strip()
        if not self.is_src:
            return ViTokenizer.tokenize(text).split()
        return text.split()

    def encode(self, text, max_len=None):
        tokens = self.tokenize(text)
        # TRUNCATION: Cắt ngắn nếu quá dài
        if max_len is not None and len(tokens) > max_len - 2: # -2 cho SOS và EOS
            tokens = tokens[:max_len-2]

        return [1] + [self.token2idx.get(t, 3) for t in tokens] + [2]

    def decode(self, indices):
        tokens = []
        for idx in indices:
            if isinstance(idx, torch.Tensor): idx = idx.item()
            if idx == 2: break
            if idx in [0, 1, 3]: continue
            tokens.append(self.idx2token.get(idx, "<UNK>"))
        return " ".join(tokens).replace("_", " ")

    def __len__(self):
        return len(self.token2idx)

class PhoMTDataset(Dataset):
    def __init__(self, hf_dataset, src_vocab=None, tgt_vocab=None):
        self.data = hf_dataset
        self.src_key = 'en' if 'en' in self.data.column_names else 'src'
        self.tgt_key = 'vi' if 'vi' in self.data.column_names else 'tgt'

        if src_vocab is None:
            self.src_vocab = Vocab(self.data, self.src_key, is_src=True)
            self.tgt_vocab = Vocab(self.data, self.tgt_key, is_src=False)
        else:
            self.src_vocab = src_vocab
            self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        src_text = item[self.src_key]
        tgt_text = item[self.tgt_key]
        # ÁP DỤNG MAX LENGTH TẠI ĐÂY
        src_encoded = self.src_vocab.encode(src_text, max_len=MAX_SEQ_LEN)
        tgt_encoded = self.tgt_vocab.encode(tgt_text, max_len=MAX_SEQ_LEN)
        return torch.tensor(src_encoded), torch.tensor(tgt_encoded)

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_pad = pad_sequence(src_batch, padding_value=0, batch_first=True)
    tgt_pad = pad_sequence(tgt_batch, padding_value=0, batch_first=True)
    return src_pad, tgt_pad

In [5]:
print("Đang tải dataset ura-hcmut/PhoMT từ Hugging Face...")
dataset = load_dataset("ura-hcmut/PhoMT", "default")

Đang tải dataset ura-hcmut/PhoMT từ Hugging Face...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/653 [00:00<?, ?B/s]

PhoMT_training.csv:   0%|          | 0.00/525M [00:00<?, ?B/s]

PhoMT_validation.csv: 0.00B [00:00, ?B/s]

PhoMT_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2977999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/18720 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19151 [00:00<?, ? examples/s]

In [6]:
train_data_full = dataset['train']
valid_data_full = dataset['validation']

In [7]:
print(f"Số lượng mẫu Train: {len(train_data_full)}")
print(f"Số lượng mẫu Validation: {len(valid_data_full)}")

Số lượng mẫu Train: 2977999
Số lượng mẫu Validation: 18720


In [8]:
print("Đang xử lý dữ liệu...")

train_dataset = PhoMTDataset(train_data_full)
valid_dataset = PhoMTDataset(valid_data_full, src_vocab=train_dataset.src_vocab, tgt_vocab=train_dataset.tgt_vocab)

Đang xử lý dữ liệu...
Đang xây dựng từ điển (Tiếng Anh)...
-> Hoàn tất. Kích thước Vocab: 20004
Đang xây dựng từ điển (Tiếng Việt)...
-> Hoàn tất. Kích thước Vocab: 20004


In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)

# MODEL

In [10]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=3, dropout=0.5):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.lstm(embedded)
        return outputs, hidden, cell

class DecoderBasic(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=3, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_token, hidden, cell, encoder_outputs=None):
        embedded = self.dropout(self.embedding(input_token))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query.unsqueeze(1)) + self.Ua(keys)))
        weights = torch.softmax(scores, dim=1)
        context = torch.bmm(weights.transpose(1, 2), keys)
        return context, weights

class DecoderBahdanau(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=3, dropout=0.5):
        super().__init__()
        self.attention = BahdanauAttention(hidden_size)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_dim + hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_token, hidden, cell, encoder_outputs):
        embedded = self.dropout(self.embedding(input_token))
        query = hidden[-1]
        context, _ = self.attention(query, encoder_outputs)
        rnn_input = torch.cat((embedded, context), dim=2)
        output, (hidden, cell) = self.lstm(rnn_input, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

class LuongAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.W = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, keys):
        keys_proj = self.W(keys)
        scores = torch.bmm(query, keys_proj.transpose(1, 2))
        weights = torch.softmax(scores, dim=2)
        context = torch.bmm(weights, keys)
        return context, weights

class DecoderLuong(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=3, dropout=0.5):
        super().__init__()
        self.attention = LuongAttention(hidden_size)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_concat = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_token, hidden, cell, encoder_outputs):
        embedded = self.dropout(self.embedding(input_token))
        lstm_out, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        context, _ = self.attention(lstm_out, encoder_outputs)
        concat_input = torch.cat((lstm_out, context), dim=2)
        concat_output = torch.tanh(self.fc_concat(concat_input))
        prediction = self.fc_out(concat_output.squeeze(1))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        vocab_size = self.decoder.fc_out.out_features
        outputs = torch.zeros(batch_size, tgt_len, vocab_size).to(self.device)
        encoder_outputs, hidden, cell = self.encoder(src)
        input_token = tgt[:, 0].unsqueeze(1)

        for t in range(1, tgt_len):
            if isinstance(self.decoder, DecoderBasic):
                prediction, hidden, cell = self.decoder(input_token, hidden, cell)
            else:
                prediction, hidden, cell = self.decoder(input_token, hidden, cell, encoder_outputs)
            outputs[:, t] = prediction
            teacher_force = random.random() < teacher_forcing_ratio
            top_token = prediction.argmax(1).unsqueeze(1)
            input_token = tgt[:, t].unsqueeze(1) if teacher_force else top_token
        return outputs

# TRAINING

In [14]:
def train_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    # Clean memory trước khi train
    torch.cuda.empty_cache()
    gc.collect()

    for i, (src, tgt) in enumerate(iterator):
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, tgt)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)
        loss = criterion(output, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

        if i % 100 == 0:
            print(f"Step {i}/{len(iterator)} | Loss: {loss.item():.4f}")

    return epoch_loss / len(iterator)

def evaluate_rouge(model, iterator, vocab_tgt):
    model.eval()
    rouge = ROUGEScore()
    preds_text = []
    targets_text = []

    with torch.no_grad():
        for i, (src, tgt) in enumerate(iterator):
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt, teacher_forcing_ratio=0)
            output_indices = output.argmax(2).tolist()
            tgt_indices = tgt.tolist()
            for j in range(len(output_indices)):
                p = vocab_tgt.decode(output_indices[j])
                t = vocab_tgt.decode(tgt_indices[j])
                preds_text.append(p)
                targets_text.append(t)
            if i > 50: break # Chỉ eval mẫu để tránh mất thời gian

    scores = rouge(preds_text, targets_text)
    return scores['rougeL_fmeasure'].item()

def run_experiment(task_name):
    # Cấu hình
    ENC_EMB = 256
    DEC_EMB = 256
    HID_DIM = 256
    N_LAYERS = 3
    ENC_DROP = 0.5
    DEC_DROP = 0.5
    LR = 0.001
    N_EPOCHS = 1 # Dù để 1 epoch nhưng nếu lâu quá bạn cứ bấm Stop

    INPUT_DIM = len(train_dataset.src_vocab)
    OUTPUT_DIM = len(train_dataset.tgt_vocab)

    # Dọn dẹp bộ nhớ
    torch.cuda.empty_cache()
    gc.collect()

    print(f"\n{'='*20}\nBẮT ĐẦU: {task_name}\n{'='*20}")

    # Khởi tạo model
    enc = Encoder(INPUT_DIM, ENC_EMB, HID_DIM, N_LAYERS, ENC_DROP)

    if task_name == 'Bai 1 (Basic)':
        dec = DecoderBasic(OUTPUT_DIM, DEC_EMB, HID_DIM, N_LAYERS, DEC_DROP)
    elif task_name == 'Bai 2 (Bahdanau)':
        dec = DecoderBahdanau(OUTPUT_DIM, DEC_EMB, HID_DIM, N_LAYERS, DEC_DROP)
    elif task_name == 'Bai 3 (Luong)':
        dec = DecoderLuong(OUTPUT_DIM, DEC_EMB, HID_DIM, N_LAYERS, DEC_DROP)

    model = Seq2Seq(enc, dec, device).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    # --- KHỐI LỆNH TRY-EXCEPT ĐỂ BẮT SỰ KIỆN DỪNG ---
    try:
        for epoch in range(N_EPOCHS):
            start_time = time.time()
            print(f"Epoch {epoch+1} đang chạy... (Bấm Stop để dừng sớm và giữ model)")

            # Train loop
            train_loss = train_epoch(model, train_loader, optimizer, criterion, 1)

            end_time = time.time()
            print(f'Epoch: {epoch+1:02} | Time: {end_time - start_time:.0f}s | Train Loss: {train_loss:.3f}')

    except KeyboardInterrupt:
        print("\n\n>>> ĐÃ DỪNG THỦ CÔNG! (Bạn đã bấm nút Stop)")
        print(">>> Đang lưu lại trạng thái Model hiện tại để đánh giá...")

    # Đánh giá model
    print("\nĐang đánh giá ROUGE-L trên tập Validation...")
    rouge_l = evaluate_rouge(model, valid_loader, train_dataset.tgt_vocab)
    print(f'>> KẾT QUẢ CUỐI CÙNG {task_name} - ROUGE-L: {rouge_l:.4f}')

    return model

# MAIN EXECUTION

In [15]:
import gc

run_experiment('Bai 1 (Basic)')


BẮT ĐẦU: Bai 1 (Basic)
Epoch 1 đang chạy... (Bấm Stop để dừng sớm và giữ model)
Step 0/93063 | Loss: 9.9070
Step 100/93063 | Loss: 6.3993
Step 200/93063 | Loss: 6.4166
Step 300/93063 | Loss: 6.5899
Step 400/93063 | Loss: 6.3035
Step 500/93063 | Loss: 6.5030
Step 600/93063 | Loss: 6.4061
Step 700/93063 | Loss: 6.1951
Step 800/93063 | Loss: 6.4945
Step 900/93063 | Loss: 6.2327
Step 1000/93063 | Loss: 6.4829
Step 1100/93063 | Loss: 6.7001
Step 1200/93063 | Loss: 5.9909
Step 1300/93063 | Loss: 5.6619


>>> ĐÃ DỪNG THỦ CÔNG! (Bạn đã bấm nút Stop)
>>> Đang lưu lại trạng thái Model hiện tại để đánh giá...

Đang đánh giá ROUGE-L trên tập Validation...
>> KẾT QUẢ CUỐI CÙNG Bai 1 (Basic) - ROUGE-L: 0.0895


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(20004, 256, padding_idx=0)
    (dropout): Dropout(p=0.5, inplace=False)
    (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.5)
  )
  (decoder): DecoderBasic(
    (embedding): Embedding(20004, 256, padding_idx=0)
    (dropout): Dropout(p=0.5, inplace=False)
    (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=20004, bias=True)
  )
)

In [16]:
run_experiment('Bai 2 (Bahdanau)')


BẮT ĐẦU: Bai 2 (Bahdanau)
Epoch 1 đang chạy... (Bấm Stop để dừng sớm và giữ model)
Step 0/93063 | Loss: 9.9109
Step 100/93063 | Loss: 6.9376
Step 200/93063 | Loss: 6.6593
Step 300/93063 | Loss: 6.2824
Step 400/93063 | Loss: 6.3002
Step 500/93063 | Loss: 6.2581
Step 600/93063 | Loss: 6.2448
Step 700/93063 | Loss: 6.1673
Step 800/93063 | Loss: 6.1023
Step 900/93063 | Loss: 6.0474
Step 1000/93063 | Loss: 6.1942
Step 1100/93063 | Loss: 6.0646
Step 1200/93063 | Loss: 6.3949
Step 1300/93063 | Loss: 5.7638
Step 1400/93063 | Loss: 6.2337
Step 1500/93063 | Loss: 5.9587
Step 1600/93063 | Loss: 6.1530
Step 1700/93063 | Loss: 5.8301
Step 1800/93063 | Loss: 6.3031
Step 1900/93063 | Loss: 5.8662


>>> ĐÃ DỪNG THỦ CÔNG! (Bạn đã bấm nút Stop)
>>> Đang lưu lại trạng thái Model hiện tại để đánh giá...

Đang đánh giá ROUGE-L trên tập Validation...
>> KẾT QUẢ CUỐI CÙNG Bai 2 (Bahdanau) - ROUGE-L: 0.1054


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(20004, 256, padding_idx=0)
    (dropout): Dropout(p=0.5, inplace=False)
    (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.5)
  )
  (decoder): DecoderBahdanau(
    (attention): BahdanauAttention(
      (Wa): Linear(in_features=256, out_features=256, bias=True)
      (Ua): Linear(in_features=256, out_features=256, bias=True)
      (Va): Linear(in_features=256, out_features=1, bias=True)
    )
    (embedding): Embedding(20004, 256, padding_idx=0)
    (dropout): Dropout(p=0.5, inplace=False)
    (lstm): LSTM(512, 256, num_layers=3, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=20004, bias=True)
  )
)

In [17]:
run_experiment('Bai 3 (Luong)')


BẮT ĐẦU: Bai 3 (Luong)
Epoch 1 đang chạy... (Bấm Stop để dừng sớm và giữ model)
Step 0/93063 | Loss: 9.9047
Step 100/93063 | Loss: 6.8784
Step 200/93063 | Loss: 6.8815
Step 300/93063 | Loss: 6.8494
Step 400/93063 | Loss: 6.4752
Step 500/93063 | Loss: 6.6144
Step 600/93063 | Loss: 6.4051
Step 700/93063 | Loss: 6.3723
Step 800/93063 | Loss: 6.1704
Step 900/93063 | Loss: 6.6011
Step 1000/93063 | Loss: 6.0856
Step 1100/93063 | Loss: 6.0742
Step 1200/93063 | Loss: 5.8294
Step 1300/93063 | Loss: 6.2705
Step 1400/93063 | Loss: 6.1717
Step 1500/93063 | Loss: 5.9637
Step 1600/93063 | Loss: 5.8998
Step 1700/93063 | Loss: 6.2523
Step 1800/93063 | Loss: 6.1026
Step 1900/93063 | Loss: 5.8351
Step 2000/93063 | Loss: 5.9535
Step 2100/93063 | Loss: 5.8655
Step 2200/93063 | Loss: 5.9256
Step 2300/93063 | Loss: 5.9603
Step 2400/93063 | Loss: 5.7793
Step 2500/93063 | Loss: 5.5674
Step 2600/93063 | Loss: 5.9427


>>> ĐÃ DỪNG THỦ CÔNG! (Bạn đã bấm nút Stop)
>>> Đang lưu lại trạng thái Model hiện tại để đá

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(20004, 256, padding_idx=0)
    (dropout): Dropout(p=0.5, inplace=False)
    (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.5)
  )
  (decoder): DecoderLuong(
    (attention): LuongAttention(
      (W): Linear(in_features=256, out_features=256, bias=True)
    )
    (embedding): Embedding(20004, 256, padding_idx=0)
    (dropout): Dropout(p=0.5, inplace=False)
    (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.5)
    (fc_concat): Linear(in_features=512, out_features=256, bias=True)
    (fc_out): Linear(in_features=256, out_features=20004, bias=True)
  )
)